# Detection & Recognition using PaddlePaddle OCR v3 basemodel

In [13]:
from paddleocr import PaddleOCR,draw_ocr
from IPython.display import display

ocr = PaddleOCR(lang='en') # need to run only once to download and load model into memory
img_path = r"C:\Users\user\Desktop\4929bf687f69997c62cfbe7265845493.jpg"
# img_path = r"C:\Users\VIGNESH SHARMA\Desktop\digilocker format AADHAR.jpg"
# img_path = r"C:\Users\VIGNESH SHARMA\Desktop\APPLICATION DOCUMENTS\AADHAAR CARD.jpg"
result = ocr.ocr(img_path, cls=False)
for idx in range(len(result)):
    res = result[idx]
    for line in res:
        print(line)

# draw result
from PIL import Image
result = result[0]
image = Image.open(img_path).convert('RGB')
boxes = [line[0] for line in result]
txts = [line[1][0] for line in result]
scores = [line[1][1] for line in result]
# im_show = draw_ocr(image, boxes, txts, scores, font_path=r"C:\Users\VIGNESH SHARMA\Desktop\simfang.ttf")
# im_show = Image.fromarray(im_show)

# result_path = 'result.jpg'
# im_show.save(result_path)
# result = Image.open(result_path)
# display(result)
# result.show()

[2024/05/10 00:22:37] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\user/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\user/.paddleocr/whl\\rec\\en\\en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_lengt

In [14]:
newtext = []
newcoord = []

for i in range(len(txts)):
    if(scores[i] >= 0.7):
        newtext.append(txts[i])
        newcoord.append(boxes[i][0])
print(newtext)
print(newcoord)

['Government of India', '/EnrollmentNo.1062/12805/41923', 'To', 'Sudhir Kumar', 'S/O Raj Kishor Saw', '20', 'dhantoliya mohlla pani tanki ward no-26', 'Dehri', 'Dehri-on-sone', 'Dehri Rohtas', 'Bihar 821307', '96', '9135620202', '2201', 'MD942007962FH', '3 3 /Your Aadhaar No.', '9757 08603794', '3', 'Government of India', 'Sudhir Kumar', '/DB14/12/1999', '/Male', '9757 0860 3794', '3']
[[200.0, 235.0], [167.0, 272.0], [208.0, 367.0], [208.0, 402.0], [208.0, 420.0], [190.0, 433.0], [208.0, 442.0], [207.0, 460.0], [207.0, 477.0], [209.0, 495.0], [207.0, 516.0], [191.0, 526.0], [208.0, 535.0], [191.0, 545.0], [235.0, 583.0], [146.0, 693.0], [200.0, 732.0], [198.0, 786.0], [271.0, 879.0], [251.0, 927.0], [251.0, 947.0], [252.0, 969.0], [261.0, 1087.0], [197.0, 1121.0]]


In [25]:
start_index = 0
end_index = 0
name_flag = False
state_flag = False
for i in range(len(newtext)):
    if("Enrol" in newtext[i]):
        start_index = i+1
        name_flag = True
    if(newtext[i].isdigit() and len(newtext[i])>=10):
        j = i-1
        while(len(newtext[j])<6):
            j-=1
        end_index = j
        state_flag = True
    if(state_flag and name_flag):
        break
print(start_index,end_index)
    

2 10


In [27]:
llimit = newcoord[end_index][0]-10
rlimit = newcoord[end_index][0]+10
temptxt = []

for i in range(len(newtext)):
    if(i<start_index):
        continue
    if(i>end_index):
        break
    if(newcoord[i][0]>=llimit and newcoord[i][0]<=rlimit):
        print(newcoord[i][0])
        temptxt.append(newtext[i])
temptxt

208.0
208.0
208.0
208.0
207.0
207.0
209.0
207.0


['To',
 'Sudhir Kumar',
 'S/O Raj Kishor Saw',
 'dhantoliya mohlla pani tanki ward no-26',
 'Dehri',
 'Dehri-on-sone',
 'Dehri Rohtas',
 'Bihar 821307']

In [28]:
name = ""
address = ""
state = ""
# pincode = temptxt[len(temptxt)-1][len(temptxt)-6:
flag = False
for word in temptxt:
    if(word == "To" or word == "To," or word == "To " or word == "TO." or word == "TO," or word == "TO "):
        continue
    if((word == name) or (name != "" and (name[0:name.index(" ")] in word))):
        continue
    if(name == ""):
        name = word
        continue
    if(address == ""):
        address+= word
        flag = True
        continue
    if(flag):
        address+=', '
        address+=word
print(name)
print(address)

Sudhir Kumar
S/O Raj Kishor Saw, dhantoliya mohlla pani tanki ward no-26, Dehri, Dehri-on-sone, Dehri Rohtas, Bihar 821307


In [7]:
from flask import Flask
from flask import request
from flask import render_template
import stringComparison

app = Flask(_name_)

@app.route('/')
def my_form():
    return render_template("my-form.html") # This should be the name of your HTML file

@app.route('/', methods=['POST'])
def my_form_post():
    text1 = request.form['text1']
    text2 = request.form['text2']
    plagiarismPercent = stringComparison.extremelySimplePlagiarismChecker(text1,text2)
    if plagiarismPercent > 50 :
        return "<h1>Plagiarism Detected !</h1>"
    else :
        return "<h1>No Plagiarism Detected !</h1>"

if _name_ == '_main_':
    app.run()

SyntaxError: invalid non-printable character U+00A0 (2042482317.py, line 23)

# Detection & Recognition using IRAS Pricetag OCR

In [9]:
# from paddleocr import PaddleOCR,draw_ocr

# det_dir = '/Users/a0g0dl6/Documents/Projects/paddleocr/models/det/'
# rec_dir = '/Users/a0g0dl6/Documents/Projects/paddleocr/models/rec/'
# rec_char_dict_path = '/Users/a0g0dl6/Documents/Projects/paddleocr/models/EN_symbol_dict.txt'

# ocr = PaddleOCR(det_model_dir=det_dir, rec_model_dir=rec_dir)

# img_path = '/Users/a0g0dl6/Documents/Projects/paddleocr/aadhaar.jpg'
# result = ocr.ocr(img_path)
# for idx in range(len(result)):
#     res = result[idx]
#     for line in res:
#         print(line)

# # draw result
# from PIL import Image
# result = result[0]
# image = Image.open(img_path).convert('RGB')
# boxes = [line[0] for line in result]
# txts = [line[1][0] for line in result]
# scores = [line[1][1] for line in result]
# im_show = draw_ocr(image, boxes, txts, scores, font_path='/Users/a0g0dl6/Documents/Projects/paddleocr/simfang.ttf')
# im_show = Image.fromarray(im_show)
# im_show.save('result_iras_ocr.jpg')


